In [ ]:
import xarray as xr

In [ ]:
# Fetch the dataset

reference_filesystem = (
    "https://storage.googleapis.com/pan3d-public-01/pfsmall/pfsmall-gcp.json"
)
xrdata = xr.open_dataset(
    "reference://",
    engine="zarr",
    mask_and_scale=False,
    backend_kwargs={
        "consolidated": False,
        "storage_options": {
            "fo": reference_filesystem,
            "asynchronous": False,  # Set for reference fs
            "target_options": {
                "asynchronous": False  # Set for target fs (HTTP in this case)
            },
        },
    },
)
xrdata

In [ ]:
# Instantiate the VTK filter to be added to the explorer/viewer's pipeline
from vtkmodules.vtkFiltersCore import vtkThreshold


threshold = vtkThreshold()
# We need to exclude Material IDs == 0, so keeping all from 1 to 15
threshold.SetLowerThreshold(1)
threshold.SetUpperThreshold(15)
threshold.SetInputArrayToProcess(0, 0, 0, 0, "Material_ID")

In [ ]:
# Create the instance of the viewer, and pass the filter to the pipeline argument

from pan3d.viewers.preview import XArrayViewer

viewer = XArrayViewer(xarray=xrdata, pipeline=[threshold], server="preview")
await viewer.ui.ready

viewer.ui